# Lesson 10: Class-Based Views Continued
---
Intro: We will continue learning about different class-based views.

# Review
---

1. What are the differences between a class-based view and a function-based view?
2. What is a DetailView?
2. What is a ListView?

# Concept 1: CreateView
---


## Objective
Before, only the admin can create posts by going on the admin page. Let's create a view where users can create posts in the blog itself. Review the following steps.
## CreateView
1. First off, go [here](https://docs.djangoproject.com/en/3.1/ref/class-based-views/generic-editing/) to read more about the generic views.
30. Open `views.py`, and include `CreateView` with all the rest of the views.
31. `class PostCreateView(CreateView):` - Create a class called PostCreateView that inherits from CreateView. So when a user wants to create a post, they need to enter a title and the content. CreateView handles the model and the fields to create the view selected.
32. `model = Post` - The Post model handles what goes on behind the scenes
33. `fields = ['title','content']` - These input fields are necessary when creating a post.
34. Since we're finished with that part, go to the urls.py page and create a url pattern for this - `path('post/new/', PostCreateView.as_view(), name='post-create')`
35. We now need a template to go along with this. The UpdateView that we will create later will share this template so let's name it `post_form.html` and put it in `blog/templates/blog`.
36. As a starting point, copy the register page from users/ and paste it onto post_form.html.
37. Change the legend to say Blog Post instead of Join Today. Change the button to say Post instead of Sign up. Delete the entire div containing the class border-top pt-3. With all these changes, we just have the form available.
38. Now open the webiste on the browser and head to `localhost:8000/post/new/`. We can't create a post yet because we didn't set up the button nor the validation checks.
39. So head over to `views.py` and within the PostCreateView, create this function
```
def form_valid(self, form):
      form.instance.author = self.request.user
      return super().form_valid(form)
```
* `form.instance.author = self.request.user` - Set the current author logged in to be the author of the post that will be created
* `return super().form_valid(form)` - This uses the base class (CreateView's) form_valid method that simply checks if a form is valid given the current author
12. Let's run the browser again and try create a new post at `localhost:8000/post/new/`. What happens?
41. When you click submit. We get an error saying we must provide a url or define a get_absolute_url method on the Model. We will do just that.

> Difference between relative url and absolute url:
An absolute URL contains all the information necessary to locate a resource.
Ex: https://www.google.com/images/id/383/cats
A relative URL provides only the tag of an absolute URL. 
Ex: /383/cats

Click this [picture](https://images.app.goo.gl/azNRNV5xPZeDjtLTA)

## SKIP TO HERE

14. Open the blog's `models.py` and create this method:

```
from django.urls import reverse

def get_absolute_url(self):
    return reverse('post-detail', kwargs={'pk':self.pk})
```
* Reverse will return the full path as a string. The path that we want to get is the path to the post-detail route. We need a specific post with a primary key so we set our [kwargs](https://www.geeksforgeeks.org/args-kwargs-python/) value or keyword argument equal to the primary key.


15. Let's go to the browser again and create a new post. This should work now!

## Mixins

16. One more thing, we need to make sure that we are logged in to create a post. Otherwise, we will just be redirected to the login page. Since we are using class-based views we can't use decorators anymore because those are reserved for functions. We will use mixins. Mixins allow for multiple inheritances for a single class. Put it simply, mixins are basically decorators for classes. Put it even more simply, mixins provide more features.
45. Go to the views.py page and include this:
`from django.contrib.auth.mixins import LoginRequiredMixin`
46. So Python allows for multiple inheritance so in the PostCreateView on the far left of what's being passed in put `LoginRequiredMixin`. It becomes this - `class PostCreateView(LoginRequiredMixin, CreateView)`
47. Now log out, and go to `localhost:8000/post/new/`. You should be redirected to the log in page.

## DIY:
---

1. Explain CreateView. What is necessary to have when using CreateView?
2. What are mixins?

# Concept 2: UpdateView
---


## Objective
Now we need an UpdateView so we can update posts on the frontend instead of going to the admin page.
## UpdateView
20. Open views.py and include `UpdateView` when importing from `django.views.generic`.
49. The UpdateView is very similar to PostCreateView because we need to update the fields that we already have. So simply copy the `PostCreateView` class and paste what you copied beneath it.
50. Rename it to `PostUpdateView`, import from `UpdateView` instead of `CreateView`. That's it!
51. Now we need a url path so go to the urls.py page and import `PostUpdateView` from `.views`
52. Create a path that will update for each post which means we need to include the primary key. Here is the full path - `path('post/<int:pk>/update/', PostUpdateView.as_view(), name='post-update')`
* Since we are providing the primary key to the post we want to update, Django will take care of the rest to update the views.
25. Let's open the browser, log in, click on a post, then go to `localhost:8000/post/<number>/update/`. Update it and then post it.
54. So as of right now, anyone can edit the posts. We need a check where only the author of the post can edit. We are going to use another mixin called UserPassesTestMixin. Let's read about it [here](https://docs.djangoproject.com/en/3.1/topics/auth/default/#django.contrib.auth.mixins.UserPassesTestMixin).
55. Go to views.py and import `UserPassesTestMixin` from `django.contrib.auth.mixins`
56. Now go to the PostUpdateView class and pass in `UserPassesTestMixin` in the middle of the two parameters.
57. Just as mentioned in the documentation, we need to override test_func(). So within the class, create a method like this - 
```
def test_func(self):
      post = self.get_object()
      return self.request.user == post.author
```
* `post = self.get_object()` - get_object() is a method of UpdateView so use this to get the single post.
* `return self.request.user == post.author` - This checks the current user is equal to the post's author. 
30. Let's open the browser again and try edit a post from another user. If you don't have another post from someone else, log out and create a post under another user.


## DIY:
---

1. Why are `PostCreateView` and `PostUpdateView` similar?
2. What are mixins again?

# Concept 3: DeleteView
---


## Objective
As it says in the name, we need a view to delete posts. 
## DeleteView
31. Return to our views.py and import `DeleteView` from `django.views.generic`
60. `class PostDeleteView(DeleteView):` - Here we create a class called PostDeleteView that inherits from DeleteView
61. `model = Post` - we set the model to Post. 
> Do you remember why?
62. Okay, in order to delete a post we need to make sure someone is logged in and is the author of the post. These checks are simple as we just include the mixins to the parameters. Include `LoginRequiredMixin` and `UserPassesTestMixin` to the left of the `DeleteView` parameter in the PostDeleteView class.
63. Remember that we need to override the test_func() for the `UserPassesTestMixin` so we can simply copy the `test_func()` from the UpdateView and paste it within the `PostDeleteView` class.
64. You should know the next step! Next go to the urls.py page and import `PostDeleteView`.
65. Copy the PostUpdateView path and paste it beneath the PostUpdateView path. Replace `update` with `delete`
66. Now we need a template that goes with the PostDeleteView. This will be a form that askss if we're sure that we want to delete the post and if we submit the form then the post will be deleted.
67. So under `blog/templates/blog` create a file called `post_confirm_delete.html`. Simply copy the code from `post_form.html` and paste it onto the newly created file.
68. Delete the load crispy forms line at the top and also `{{form | crispy}}}`. Change the legend to say `Delete Post`. 
69. Beneath the legend create an h2 tag that says "Are you sure you want to delete the post" so it looks like this - 
```
<h2>Are you sure you want to delete the post "{{ object.title }}"</h2>
```
70. Change the button to say `Yes, delete` with a class of `btn-outline-danger` instead of info.
71. If the user changes their mind and not want to delete the post, we need to account for that as well. We will create an a tag beneath the button. 
72. Simply copy the button divs and paste it beneath itself. Change the `<button>` tags to `<a>` tags. Remove `type=submit` and add in `href="{% url 'post-detail' object.id %}"`. Change the btn-outline class to become `btn-outline-secondary`. Change the text to say Cancel.
73. Let's open the browser, make sure you are logged in, go to a post, then go to `localhost:8000/post/<number>/delete/`, press cancel and then delete.
74. We should get an error because Django doesn't know where to redirect us once we delete a post. We need to provide a success_url that goes to the home page.
75. So return to the views.py to the PostDeleteView class. Right below the model variable include this - `success_url = '/'` This just takes us right back to the home page.
76. Let's open the browser, make sure you are logged in, go to a post, then go to `localhost:8000/post/<number>/delete/`, then delete a post. 

## DIY:
---

1. To check your understanding, what do the LoginRequiredMixin and UserPassesTestMixin do?
2. What is the success_url variable?
3. Do you remember why we need to include model for our class-based views?



# Concept 4: Bringing it all together
---


## Objective
Now instead of manually typing the url paths for creating, updating, and deleting posts, we will clean it up so we can click on links.
## Navigation Bar
49. Let's put a link in our navigation bar to create a new post. Open the `base.html` and scroll down to the Navbar Right Side. To make it easier on ourselves, copy the Profile a tag and paste it right below it. Change the url to become `'post-create'` and the text from Profile to `New Post`.
78. Now add links to update and delete posts. This will be done in the `post_detail.html` where only the correct user can update and delete their posts.  
79. Right after the small tag with the date posted variable. Include this code - 
```
{% if object.author == user %}
{% endif %}
```
* This checks if the post's author is the current user.
52. Now in between the if statement include this - 
```
<div>
      <a class="btn btn-secondary btn-sm mt-1 mb-1" href={% url 'post-update' object.id %}>Update</a>
      <a class="btn btn-danger btn-sm mt-1 mb-1" href={% url 'post-delete' object.id %}>Delete</a>
</div>
```
* > Can you describe what's going on here?
53. Let's check it out in the browser again. Log in and you should see a link that takes you to create a new post. Click on a post that you created. You should see update and delete buttons.
82. We're done! Push all your code to GitHub.

## DIY:
---

1. What does `class="btn btn-danger` display?
2. What is `object.id`?

# Summary:
---


1. What are class-based views?
2. What are function-based views?
3. Explain ListView
3. Explain DetailView
3. Explain CreateView
3. Explain UpdateView
3. Explain DeleteView

# Homework:
---



1. Continue working on the BlackJack game.

# Notes on homework:
---

I will check in on Thursday,  through email to check on your progress. Respond with any questions you might have. Otherwise, a simple “all good” is appropriate if you have no questions or comments. 

You will need to upload your coding homework assignments to GitHub.
1. In gitbash, change directories to the homework directory: tomas_python/homework
* TIP: use ‘cd’ to change directories
* Use ‘cd ..’ to return to the previous directory
* Use ‘pwd’ to show full pathname of the current working directory 
* Use ‘ls’ to list all your directories
2. Once you’re in that directory, type in ‘git pull’
* This ensures you have all updated files
* If there is an error involved, email me immediately so we can try resolving it.
* Otherwise, type your code below and we’ll resolve issues next class
3. To create a new file, type in ‘touch hw01.py’ or the appropriate file name
* ‘Touch’ creates a new file
4. Open up the python file and start coding!

Note: Become familiar with these actions. This is essentially what happens in the backend when you right-click and create a new folder/file!
